1. Load the packages

In [ ]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torch.utils.data import Dataset,DataLoader,TensorDataset
from torch.autograd.function import Function
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torchvision import datasets, transforms
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data as data
from PIL import Image
import os

2. Load the dataset

In [ ]:
num_workers=2
image_size=28
path='.\data'
def get_loader():
    
    transform_usps = transforms.Compose([
                    transforms.Scale(image_size),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5, ), (0.5, ))])
    transform_mnist=  transforms.Compose([
                    transforms.Scale(image_size),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5,), (0.5,))])
    
    usps_train = datasets.USPS(root=path,train=True,download=True, transform=transform_usps)
    usps_test = datasets.USPS(root=path,train=False, download=True, transform=transform_usps)

    mnist_train = datasets.MNIST(root=path, train=True,download=True, transform=transform_mnist)
    mnist_test = datasets.MNIST(root=path, train=False,download=True, transform=transform_mnist)


    return usps_train, usps_test,mnist_train,mnist_test

In [ ]:
usps_train, usps_test,mnist_train,mnist_test=get_loader()

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:317: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


  0%|          | 0/6579383 [00:00<?, ?it/s]

  0%|          | 0/1831726 [00:00<?, ?it/s]

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting .\data/MNIST/raw/train-images-idx3-ubyte.gz to .\data/MNIST/raw



...

Extracting .\data/MNIST/raw/train-labels-idx1-ubyte.gz to .\data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting .\data/MNIST/raw/t10k-images-idx3-ubyte.gz to .\data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting .\data/MNIST/raw/t10k-labels-idx1-ubyte.gz to .\data/MNIST/raw



In [ ]:
print(len(mnist_train))
print(len(mnist_test))
print(len(usps_train))
print(len(usps_test))

60000
10000
7291
2007


3. Define function for randomly selecting samples from source and target domains

In [ ]:

def get_index(labels,select_num):
  select_list=np.array([])
  remaining_list=np.array([])
  for i in range(10):
    full_list=np.where(labels==i)[0]
    ran_select=np.random.choice(full_list,select_num,replace=False)
    select_list=np.hstack((select_list,ran_select))

    remain_list=np.setdiff1d(full_list,ran_select)
    remaining_list=np.hstack((remaining_list,remain_list))

  return select_list.astype(int),remaining_list.astype(int)

In [ ]:
def decompose_dataset(datesett):
  
  x_set=[]
  y=[]
  for i in range(len(datesett)):
    x,tem_y=datesett[i]
    x_set.append(x.numpy())
    y.append(tem_y)

  return np.array(x_set),np.array(y)
 


4. Unweighted center transfer layer

In [ ]:
class UCTL(nn.Module):
    def __init__(self, num_classes, feat_dim, size_average=True):
        super(UCTL, self).__init__()
        self.centers = nn.Parameter(torch.randn(num_classes, feat_dim))
        self.centerlossfunc = CenterlossFunc.apply
        self.feat_dim = feat_dim
        self.size_average = size_average

    def forward(self, label, flip_label, feat):
        batch_size = feat.size(0)
        feat = feat.view(batch_size, -1)
        # To check the dim of centers and features
        if feat.size(1) != self.feat_dim:
            raise ValueError("Center's dim: {0} should be equal to input feature's \
                            dim: {1}".format(self.feat_dim,feat.size(1)))
        batch_size_tensor = feat.new_empty(1).fill_(batch_size if self.size_average else 1)
        loss = self.centerlossfunc(feat, label,flip_label, self.centers, batch_size_tensor)
        return loss


class CenterlossFunc(Function):
    @staticmethod
    def forward(ctx, feature, label,flip_label, centers, batch_size):
        ctx.save_for_backward(feature, label,flip_label, centers, batch_size)
        centers_batch = centers.index_select(0, flip_label.long())
        return (feature - centers_batch).pow(2).sum() / (2.0 * batch_size)

    @staticmethod
    def backward(ctx, grad_output):
        feature,label,flip_label,centers,batch_size = ctx.saved_tensors
        flip_centers_batch = centers.index_select(0, flip_label.long())
        flip_diff=flip_centers_batch-feature


        centers_batch = centers.index_select(0, label.long())
        diff = centers_batch - feature
        # init every iteration
        counts = centers.new_ones(centers.size(0))
        ones = centers.new_ones(label.size(0))
        grad_centers = centers.new_zeros(centers.size())
        counts = counts.scatter_add_(0, label.long(), ones)-1+0.0001

        grad_centers.scatter_add_(0, label.unsqueeze(1).expand(feature.size()).long(), diff)
        grad_centers = grad_centers/counts.view(-1, 1)



        return - grad_output * flip_diff / batch_size, None, None,grad_centers / batch_size, None

#Testing function
def main(test_cuda=False):
    print('-'*80)
    device = torch.device("cuda" if test_cuda else "cpu")
    ct = UCTL(20,2,size_average=True).to(device)
    y = torch.Tensor([0,0,2,1,3]).to(device)
    feat = torch.zeros(5,2).to(device).requires_grad_()
    print(list(ct.parameters()))
    
    print(ct.centers.grad)
    out = ct(y,y,feat)
    print(out.item())
    out.backward()


if __name__ == '__main__':
    torch.manual_seed(999)
    if torch.cuda.is_available():
        main(test_cuda=True)

--------------------------------------------------------------------------------
[Parameter containing:
tensor([[-0.2528,  1.4072],
        [ 0.2910,  1.0365],
        [-0.9816, -3.4219],
        [ 1.4910,  0.2422],
        [ 1.4832, -0.3704],
        [ 0.0941,  2.1528],
        [ 0.6271, -1.1666],
        [-0.7862,  0.0759],
        [-0.0086, -0.6568],
        [-1.0011,  0.2992],
        [ 0.6396, -1.0857],
        [-1.6153,  1.5635],
        [ 0.8194,  0.6117],
        [ 0.7602,  1.4788],
        [ 1.9647,  0.9414],
        [ 0.3883, -0.3957],
        [ 0.5920, -2.8563],
        [-0.4750, -0.9978],
        [ 0.0489,  0.9250],
        [-1.2278, -0.9470]], device='cuda:0', requires_grad=True)]
None
2.02020525932312


5. Define LetNet++

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1_1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.prelu1_1 = nn.PReLU()
        self.conv1_2 = nn.Conv2d(32, 32, kernel_size=5, padding=2)
        self.prelu1_2 = nn.PReLU()
        self.conv2_1 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.prelu2_1 = nn.PReLU()
        self.conv2_2 = nn.Conv2d(64, 64, kernel_size=5, padding=2)
        self.prelu2_2 = nn.PReLU()
        self.conv3_1 = nn.Conv2d(64, 128, kernel_size=5, padding=2)
        self.prelu3_1 = nn.PReLU()
        self.conv3_2 = nn.Conv2d(128, 128, kernel_size=5, padding=2)
        self.prelu3_2 = nn.PReLU()
        self.preluip1 = nn.PReLU()
        self.ip1 = nn.Linear(int(2304/2), 256)
        self.ip2 = nn.Linear(256, 10, bias=True)


    def forward(self, x):
        x = self.prelu1_1(self.conv1_1(x))
        x = self.prelu1_2(self.conv1_2(x))
        x = F.max_pool2d(x,2)
        x = self.prelu2_1(self.conv2_1(x))
        x = self.prelu2_2(self.conv2_2(x))
        x = F.max_pool2d(x,2)
        x = self.prelu3_1(self.conv3_1(x))
        x = self.prelu3_2(self.conv3_2(x))
        x = F.max_pool2d(x,2)
        x = x.view(-1, int(2304/2))

        ip1 = self.preluip1(self.ip1(x))
        ip2 = self.ip2(ip1)
        return ip1, F.log_softmax(ip2, dim=1)


In [ ]:
from torchsummary import summary
device = torch.device("cuda")
net = Net().to(device)

summary(net, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             832
             PReLU-2           [-1, 32, 28, 28]               1
            Conv2d-3           [-1, 32, 28, 28]          25,632
             PReLU-4           [-1, 32, 28, 28]               1
            Conv2d-5           [-1, 64, 14, 14]          51,264
             PReLU-6           [-1, 64, 14, 14]               1
            Conv2d-7           [-1, 64, 14, 14]         102,464
             PReLU-8           [-1, 64, 14, 14]               1
            Conv2d-9            [-1, 128, 7, 7]         204,928
            PReLU-10            [-1, 128, 7, 7]               1
           Conv2d-11            [-1, 128, 7, 7]         409,728
            PReLU-12            [-1, 128, 7, 7]               1
           Linear-13                  [-1, 256]         295,168
            PReLU-14                  [

6. Define the training in one epoch

In [ ]:
#deine the training within 1 epoch
test_cuda=True
device = torch.device("cuda" if test_cuda else "cpu")
def train(epoch,loss_weight,train_loader,model,nllloss,uctl,optimizer4nn,optimzer4center,test_loader):
  # 
  ip1_loader = []
  idx_loader = []
  center_total=0
  nll_losss=0
  #Training in each epoch#
  for data, target,good_ord,flip_ord in train_loader:
    data, target,good_ord,flip_ord = data.to(device), target.to(device),good_ord.to(device), flip_ord.to(device)
    ip1, pred = model(data)

    loss = nllloss(pred, target) + loss_weight* uctl(good_ord, flip_ord,ip1)

    optimizer4nn.zero_grad()
    optimzer4center.zero_grad()

    loss.backward()

    optimizer4nn.step()
    optimzer4center.step()

    ip1_loader.append(ip1)
    idx_loader.append((target))
    center_total=center_total+uctl(good_ord, flip_ord,ip1).item()
    nll_losss= nll_losss+nllloss(pred, target).item()*64
  model.eval()
  correct = 0
  for inputs, labels in test_loader:
    inputs, labels= inputs.cuda(),labels.cuda()
    ip1, outputs = model(inputs)
    preds = torch.max(outputs, 1)[1]
    correct += torch.sum(preds == labels.data)
  epoch_acc = correct.double() / len(test_loader.dataset)
  if epoch>45:
    print("Training... Epoch = %d" % epoch)
    print('Softmax:', nll_losss/len(train_loader.dataset))
    print('UCTL:', center_total/len(train_loader))
    print('Testing acc:',epoch_acc.item())
  return model,epoch_acc



7. Define the training function

In [ ]:
def trainss(direction,weightss,center_step):


  src_label = np.array(mnist_train.targets)
  tar_label = np.array(usps_train.targets)
  srctrain_index,_=get_index(src_label,5000)
  source_trainset=torch.utils.data.Subset(mnist_train, srctrain_index)
  source_trainset=source_trainset
  tartrain_index,tartest_index=get_index(tar_label,10)
  target_trainset=torch.utils.data.Subset(usps_train, tartrain_index)
  train_set=torch.utils.data.ConcatDataset([source_trainset, target_trainset])
  test_set = usps_test
  if direction=='MNISTtoUSPS':
    src_label = np.array(mnist_train.targets)
    tar_label = np.array(usps_train.targets)
    srctrain_index,_=get_index(src_label,5000)
    source_trainset=torch.utils.data.Subset(mnist_train, srctrain_index)
    source_trainset=source_trainset
    tartrain_index,tartest_index=get_index(tar_label,10)
    target_trainset=torch.utils.data.Subset(usps_train, tartrain_index)
    train_set=torch.utils.data.ConcatDataset([source_trainset, target_trainset])
    train_x,train_y=decompose_dataset(train_set)
    train_x=torch.from_numpy(train_x)
    train_x=train_x.float()
    good_order=np.hstack((train_y[:-100],train_y[-100:]+10))

    flip_order=np.hstack((train_y[:-100]+10,train_y[-100:]))
    good_order=torch.from_numpy(good_order)
    flip_order=torch.from_numpy(flip_order)


    good_order=good_order.long()
    flip_order=flip_order.long()

    train_y=torch.from_numpy(train_y)
    train_y=train_y.long()

    batch_size=64
    train_dataset=TensorDataset(train_x,train_y,good_order,flip_order)

    train_loader= DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    print(len(train_set))
  


    test_set = usps_test
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)
    print(len(test_set))

  else:
    src_label = np.array(usps_train.targets)
    tar_label = np.array(mnist_train.targets)
    source_trainset=usps_train
    tartrain_index,tartest_index=get_index(tar_label,10)
    target_trainset=torch.utils.data.Subset(mnist_train, tartrain_index)
    train_set=torch.utils.data.ConcatDataset([source_trainset, target_trainset])
    train_x,train_y=decompose_dataset(train_set)
    train_x=torch.from_numpy(train_x)
    train_x=train_x.float()
    good_order=np.hstack((train_y[:-100],train_y[-100:]+10))

    flip_order=np.hstack((train_y[:-100]+10,train_y[-100:]))
    good_order=torch.from_numpy(good_order)
    flip_order=torch.from_numpy(flip_order)


    good_order=good_order.long()
    flip_order=flip_order.long()

    train_y=torch.from_numpy(train_y)
    train_y=train_y.long()

    batch_size=64
    train_dataset=TensorDataset(train_x,train_y,good_order,flip_order)
    train_loader= DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    print(len(train_set))
    test_set = mnist_test
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)
    print(len(test_set))



  # Model
  model = Net().to(device)


  # Softmax loss
  nllloss = nn.CrossEntropyLoss().to(device) #CrossEntropyLoss = log_softmax + NLLLoss

  # trade-off
  loss_weight =  weightss

  # UCTL
  uctl = UCTL(20, 256).to(device)

  # optimzer4nn
  optimizer4nn = optim.Adam(model.parameters(),lr=0.00025)

  sheduler = lr_scheduler.StepLR(optimizer4nn,20,gamma=0.8)

  # optimzer4center
  optimzer4center = torch.optim.SGD(uctl.parameters(), lr =center_step)
  best=[0]
  eps=0
  print("Task:",direction)

  for epoch in range(50):
    
    # print optimizer4nn.param_groups[0]['lr']
    model,accuracy=train(epoch+1, loss_weight,train_loader,model,nllloss,uctl,optimizer4nn,optimzer4center,test_loader)
    sheduler.step()
    if accuracy > max(best):
      best.append(accuracy)
      eps=epoch

  print("Epoch:",str(eps),"Best acc:", str(max(best)))


8. Example


Note that models will be trained from scratch. Training time should last for around 15 minutes for one repetition if you use the GPU (GeForce RTX 3090).

Although the results you get may be slightly different from the ones of the manuscript due to randomized initialization, the gap should be small.

In [ ]:
direction="MNISTtoUSPS"

lambda_para,center_step=0.01,0.5
trainss(direction,lambda_para,center_step)

print('===========================')

direction="USPStoMNIST"

lambda_para,center_step=0.01,0.5
trainss(direction,lambda_para,center_step)


50100
2007
Task: MNISTtoUSPS
Training... Epoch = 46
Softmax: 2.2095307873812006e-06
UCTL: 0.034483169294099204
Testing acc: 0.9466865969108121
Training... Epoch = 47
Softmax: 2.3996915241988386e-07
UCTL: 0.01629476586123631
Testing acc: 0.950174389636273
Training... Epoch = 48
Softmax: 1.5437701692431863e-07
UCTL: 0.011271233938629638
Testing acc: 0.9476831091180866
Training... Epoch = 49
Softmax: 9.256152764749797e-08
UCTL: 0.011052319088339388
Testing acc: 0.9531639262580966
Training... Epoch = 50
Softmax: 0.005775751914286957
UCTL: 0.3619717590856969
Testing acc: 0.9292476332835077
Epoch: 48 Best acc: tensor(0.9532, device='cuda:0', dtype=torch.float64)
7391
10000
Task: USPStoMNIST
Training... Epoch = 46
Softmax: 0.0010443013368073375
UCTL: 0.29379330678232785
Testing acc: 0.9545
Training... Epoch = 47
Softmax: 0.0010613364756849263
UCTL: 0.2725942612722002
Testing acc: 0.9529000000000001
Training... Epoch = 48
Softmax: 0.0010069404713463725
UCTL: 0.29006181426089384
Testing acc: 0.